In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import pingouin as pg
import plotly.graph_objects as go
from os.path import join as pjoin
from natsort import natsorted

sys.path.append("../../")
import circletrack_behavior as ctb
import plotting_functions as pf

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
## Settings
parent_dir = 'CircleTrack_OneShot'
experiment_dir = 'OneShot1'
lin_path = f'../../../{parent_dir}/{experiment_dir}/output/lin_behav/'
circle_path = f'../../../{parent_dir}/{experiment_dir}/output/behav/'
fig_path = f'../../../{parent_dir}/{experiment_dir}/intermediate_figures'
chance_color = 'darkgrey'
avg_color = 'midnightblue'
subject_color = 'darkgrey'
two_group_colors = ['darkgrey', 'midnightblue']
male_mice = [f'os0{x}' for x in np.arange(1, 5)]

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

### Linear track results.

In [3]:
lin_dict = {'mouse': [], 'day': [], 'rewards': [], 'group': [], 'group_two': []}
for mouse in os.listdir(lin_path):
    mouse_path = pjoin(lin_path, mouse)
    sex = 'Male' if mouse in male_mice else 'Female'
    for idx, session in enumerate(os.listdir(mouse_path)):
        lin_behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        lin_dict['mouse'].append(mouse)
        lin_dict['day'].append(idx+1)
        lin_dict['rewards'].append(np.sum(lin_behav['water']))
        lin_dict['group'].append(sex)
        lin_dict['group_two'].append(lin_behav['maze'].unique()[0])
lin_df = pd.DataFrame(lin_dict)

In [7]:
## Plot rewards across days on linear track
fig = pf.plot_behavior_across_days(lin_df, x_var='day', y_var='rewards', groupby_var=['day'], plot_transitions=None,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=500)
fig.show()
fig.write_image(pjoin(fig_path, 'linear_track_rewards.png'))

In [8]:
## Plot rewards across days on linear track for male vs female
fig = pf.plot_behavior_across_days(lin_df, x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=two_group_colors, avg_color=avg_color, expert_line=False, chance=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=500)
fig.show()
fig.write_image(pjoin(fig_path, 'linear_track_rewards_mf.png'))

### Circle track accuracy across trials.

In [9]:
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy_thresh5': [], 'rewards': [], 'probe_acc': []}
for mouse in os.listdir(circle_path):
    mouse_path = pjoin(circle_path, mouse)
    sex = 'Male' if mouse in male_mice else 'Female'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav_noprobe = behav[~behav['probe']]
        behav_probe = behav[behav['probe']]
        reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
        pc_thresh5 = ctb.lick_accuracy(behav_noprobe, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=True)
        pc_probe = ctb.lick_accuracy(behav_probe, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=False)
        circletrack_results['mouse'].append(mouse)
        circletrack_results['day'].append(idx+1)
        circletrack_results['group'].append(sex)
        circletrack_results['session'].append(np.unique(behav['session'])[0])
        circletrack_results['lick_accuracy_thresh5'].append(pc_thresh5)
        circletrack_results['rewards'].append(np.sum(behav['water']))
        circletrack_results['probe_acc'].append(pc_probe)
ct_df = pd.DataFrame(circletrack_results)

In [10]:
bin_size = 4
fig = pf.custom_graph_template(x_title='Trial', y_title='Lick Accuracy (%)', width=800, shared_y=True, rows=1, columns=2)

for mouse in ct_df['mouse'].unique():
    for day in ct_df['day'].unique():
        if day < 2:
            showlegend = True 
        else:
            showlegend = False

        if mouse not in male_mice and day == 2:
            pass 
        else:
            mdata = ct_df[(ct_df['mouse'] == mouse) & (ct_df['day'] == day)].reset_index(drop=True)
            trial_acc = np.asarray(mdata['lick_accuracy_thresh5'][0])
            binned_data = ctb.bin_data(trial_acc, bin_size=bin_size)
            x_axis = np.arange(1, trial_acc.shape[0], bin_size)
            line_color = two_group_colors[1] if mouse in male_mice else two_group_colors[0]
            fig.add_trace(go.Scatter(x=x_axis, y=binned_data, mode='lines', line_color=line_color, 
                                     name=mouse, showlegend=showlegend), row=1, col=day)
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_across_trials.png'))

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/CircleTrack_OneShot/OneShot1/../../circletrack_behavior.py:525: RuntimeWarning:

Mean of empty slice



In [11]:
## Probe accuracy for male mice, since female mice were not probed
sub_male = ct_df[ct_df['group'] == 'Male']
avg_probe = sub_male.groupby(['day'], as_index=False).agg({'probe_acc': ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='Day', y_title='Probe Accuracy (%)')
fig.add_trace(go.Scatter(x=avg_probe['day'], y=avg_probe['probe_acc']['mean'], mode='lines+markers',
                         error_y=dict(type='data', array=avg_probe['probe_acc']['sem']), line_color=avg_color))
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'probe_accuracy_males.png'))